In [6]:
pip install pandas scikit-learn peft datasets tensorboardX numba bitsandbytes

In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification, \
                         TrainingArguments, Trainer, DataCollatorWithPadding, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from datasets import load_dataset, Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, make_scorer
import os
import torch
import torch.nn as nn
import pdb

## Google drive mount

In [8]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Dataset uploading

In [9]:
DATASET_PATHS = {
    "local": {
        "train": "../../datasets/train_set.csv",
        "test": "../../datasets/test_set.csv"
    },
    "local_two": {
        "train": "train_set.csv",
        "test": "test_set.csv"
    },
    "local_three": {
        "train": "drive/MyDrive/fine_tuning/train_set.csv",
        "test": "drive/MyDrive/fine_tuning/test_set.csv"
    },

    "kaggle": {
        "train": "/kaggle/input/python-codes-time-complexity/train_set.csv",
        "test": "/kaggle/input/python-codes-time-complexity/test_set.csv"
    }
}

def upload_datasets(dataset_paths=DATASET_PATHS):
    for path in dataset_paths:
        if os.path.exists(dataset_paths[path]['train']) and os.path.exists(dataset_paths[path]['test']):
            return dataset_paths[path]['train'], dataset_paths[path]['test']

    return FileNotFoundError(f"Datasets do not exist in the current paths: {dataset_paths}")


train_set_path, test_set_path = upload_datasets()

# Metrics

### Ordering labels by Hierarchy

In [10]:
LABELS_HIERARCHY = {
    'constant': 1,
    'logn': 2,
    'linear': 3,
    'nlogn': 4,
    'quadratic': 5,
    'cubic': 6,
    'np': 7
}

N_CLASSES = len(LABELS_HIERARCHY)

# Dataset uploading

In [11]:
train_set = load_dataset("csv", data_files=train_set_path)['train']
test_set = load_dataset("csv", data_files=test_set_path)['train']

train_labels = train_set['complexity']
test_labels = test_set['complexity']

# Evaluating

### Writing the custom metric *Hierarchy Complexity Score*

In [12]:
def hc_score(y_true, y_pred, n_classes=N_CLASSES):
    assert len(y_true) == len(y_pred), f"The amount of y_true labels: {len(y_true)} does not equal to the amount of y_pred: {len(y_pred)}."

    n_samples = len(y_true)

    return (np.sum(np.abs(y_pred - y_true)) / n_classes) / n_samples

## Computing metrics

In [13]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    preds = np.argmax(logits[0], axis=-1) if isinstance(logits, tuple) else np.argmax(logits, axis=-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)
    # Calculate F-1 Macro
    f1_macro_score = f1_score(labels, preds, average='macro')
    # Calculate Hierarchy Score
    hierarchy_score = hc_score(labels, preds)

    return {
        "accuracy": accuracy,
        "f1_macro": f1_macro_score,
        "hierarchy_score": hierarchy_score
    }

# Tokenizing

## Label tokenizing

In [14]:
labelEncoder = LabelEncoder()
labelEncoder.fit(train_set['complexity'])

LabelEncoder()

## Feature tokenizing

In [15]:
def tokenize_data(samples, tokenizer):
    tokenized = tokenizer(samples['code'], truncation=True, max_length=512)
    tokenized['labels'] = labelEncoder.transform(samples['complexity'])
    return tokenized


def set_tokenizer(checkpoint):
    try:
        tokenizer = AutoTokenizer.from_pretrained(checkpoint, pad_token = "<pad>")
    except Exception as e:
        print(f"Failed to load {checkpoint}: {e}")
        checkpoint = "-".join(checkpoint.split("-")[:2])
        tokenizer = AutoTokenizer.from_pretrained(checkpoint)
        print(f"Falling back to {checkpoint}")

    X_train = train_set.map(lambda x: tokenize_data(x, tokenizer), batched=True)
    X_eval = test_set.map(lambda x: tokenize_data(x, tokenizer), batched=True)

    # Collator for batch padding
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    return tokenizer, data_collator, X_train, X_eval

# Model

## Device

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Checkpoint

In [17]:
checkpoint = "deepseek-ai/DeepSeek-Coder-V2-Lite-Base"

## Quantizing

In [18]:
# Configure 4-bit quantization
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_storage=torch.bfloat16

)

## Model loading

In [19]:
def set_model(checkpoint):
    model = AutoModel.from_pretrained(checkpoint, torch_dtype='bfloat16', num_labels=7,
                                      trust_remote_code=True, device_map='auto', quantization_config=quant_config)

    return model

## Classifier head

In [20]:
class Classifier(nn.Module):
    def __init__(self, checkpoint):
        super().__init__()
        # Initting base model
        self.base_model = set_model(checkpoint)

        # Configuring padding tokens just in case if absent
        self.config()

        self.dense = nn.Linear(in_features=self.base_model.config.hidden_size, out_features=self.base_model.config.num_labels)
        self.activation = nn.Softmax(dim=-1)

    def config(self):
        # Configuring padding token in case is absent
        self.base_model.config.pad_token_id = tokenizer.pad_token_id
        # As well, as resizing the embeddings to accomodate the new *pad* token
        self.base_model.resize_token_embeddings(len(tokenizer))

    def forward(self, inputs):
        outputs = self.base_model(inputs['input_ids'], inputs['attention_mask'])
        # Capturing the first <bos> token, which contains pooled embeddings of a full sentence (not sure)
        pooled_output = outputs.last_hidden_state[:, 0, :].to(dtype=torch.float32)
        outputs = self.dense(pooled_output)
        activations = self.activation(outputs)
        return activations

## Loading tokenizer and model

In [21]:
tokenizer, data_collator, train_set, eval_set = set_tokenizer(checkpoint)
model = Classifier(checkpoint)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/978 [00:00<?, ? examples/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## X-ray scan of tokenizer and model

In [22]:
tokenizer

LlamaTokenizerFast(name_or_path='deepseek-ai/DeepSeek-Coder-V2-Lite-Base', vocab_size=100000, model_max_length=16384, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<｜begin▁of▁sentence｜>', 'eos_token': '<｜end▁of▁sentence｜>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	100000: AddedToken("<｜begin▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	100001: AddedToken("<｜end▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	100002: AddedToken("<｜fim▁hole｜>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	100003: AddedToken("<｜fim▁begin｜>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	100004: AddedToken("<｜fim▁end｜>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	100005: AddedToken("<｜completion｜>", rstrip=False, lstr

In [23]:
model

Classifier(
  (base_model): DeepseekV2Model(
    (embed_tokens): Embedding(100019, 2048)
    (layers): ModuleList(
      (0): DeepseekV2DecoderLayer(
        (self_attn): DeepseekV2Attention(
          (q_proj): Linear4bit(in_features=2048, out_features=3072, bias=False)
          (kv_a_proj_with_mqa): Linear4bit(in_features=2048, out_features=576, bias=False)
          (kv_a_layernorm): DeepseekV2RMSNorm()
          (kv_b_proj): Linear4bit(in_features=512, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): DeepseekV2YarnRotaryEmbedding()
        )
        (mlp): DeepseekV2MLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=10944, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=10944, bias=False)
          (down_proj): Linear4bit(in_features=10944, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): DeepseekV2RMSNorm

# Hyperparameters

In [25]:
def set_training_args(checkpoint, batch_size=16):
    training_args = TrainingArguments(output_dir=f"training_results/{checkpoint}/",
                                      eval_strategy="epoch",
                                      save_strategy="epoch",
                                      logging_strategy="epoch",
                                      #learning_rate=2e-4, # Testing
                                      bf16=True,
                                      report_to='tensorboard',
                                      num_train_epochs=3,
                                      warmup_steps=100, # Testing
                                      per_device_train_batch_size=batch_size,
                                      per_device_eval_batch_size=batch_size,
                                      gradient_accumulation_steps = 8,
                                      # Testing
                                      load_best_model_at_end=True,
                                     )
    return training_args

# LoRA

#### Check module names in the model to specify them in *target_modules* param

model = set_model(checkpoint)
for name, module in model.named_modules():
    print(name)

## LoRA config

In [29]:
config = LoraConfig(
    #r=16,
    #lora_alpha=32,
    #target_modules = ["q_proj", "k_proj", 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'], # Not sure about this
    lora_dropout=0.1,
    bias='none',
    #modules_to_save=['classifier'], # Not sure about this one either
    task_type = "SEQ_CLS"
)

model = get_peft_model(model=model, peft_config=config)
model.print_trainable_parameters()

TypeError: 'method' object is not subscriptable

### Flash the drive

!rm -rf training_results

# Trainer

In [ ]:
def finetune(checkpoint):
    # Collecting
    training_args = set_training_args(checkpoint=checkpoint, batch_size=2)

    # Building
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_set,
        eval_dataset=eval_set,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # Train
    trainer.train()

    # Save metrics
    test_metrics = trainer.evaluate(eval_dataset=eval_set)
    trainer.save_metrics(split="test", metrics=test_metrics)

    return trainer

trainer = finetune(checkpoint)

# Flushing CUDA

In [ ]:
!pip install GPUtil

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

free_gpu_cache()

# Inference

In [ ]:
device = torch.cuda.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
tokenizer, data_collator, train_set, eval_set = set_tokenizer(checkpoint)


In [ ]:
def predict(inputs):
    # Tokenizing inputs
    test_sample = tokenizer(inputs, return_tensors='pt', padding=True, truncation=True)
    inputs = Dataset.from_dict({key: value.to(model.device) for key, value in test_sample.items()})

    # Predicting & decoding inputs
    preds = trainer.predict(test_dataset=inputs)
    preds = labelEncoder.inverse_transform(y=np.ravel(np.argmax(preds.predictions[0], axis=-1)))

    return preds

In [ ]:
test_sample = """
class Solution:
    def topKFrequent(self, nums: List[int], k: int) -> List[int]:
        count = {}
        for num in nums:
            count[num] = 1 + count.get(num, 0)

        arr = []
        for num, cnt in count.items():
            arr.append([cnt, num])
        arr.sort()

        res = []
        while len(res) < k:
            res.append(arr.pop()[1])
        return res
        """

predict(test_sample)